In [17]:
import numpy as np
import pandas as pd
import codecs, nltk

# Load the dataset

In [370]:
df = pd.read_csv('Fake news meta/FakeNews_with_hyper_BinaryLabel(SVM)_2.csv')
#df = pd.read_csv('FakeNewsNet/mix.csv')
#df = pd.read_csv('hyperpartisan_meta.csv')

In [371]:
df['hyperpartisan'].value_counts()

True     1904
False     364
Name: hyperpartisan, dtype: int64

In [372]:
len(df)

2268

In [373]:
df.head()

,doc_id,title,mainText,veracity,mainText_preprocessed,upper_title,upper_mainText,punct_title,punct_mainText,exclamations_title,exclamations_text,title_words_not_stopword,text_words_not_stopword,title_mainText,hyperpartisan
0,0,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,Results from a new study concluded that millio...,False,result new study conclude million could vote p...,10,0,1,39,1,0,13,204,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,True
1,1,President Trump Signed An Executive Order That...,Illegal immigrants do NOT have the same rights...,False,illegal immigrant right american citizen illeg...,0,3,1,17,0,0,12,108,President Trump Signed An Executive Order That...,True
2,2,New Document Reveals Lynch Instructed Comey to...,"Ah, the infamous tarmac meeting. You know…the...",False,ah infamous tarmac meeting one former attorney...,0,1,1,16,0,0,9,77,New Document Reveals Lynch Instructed Comey to...,True
3,3,Newly Leaked Report Shows Hillary DID NOT Win ...,President Trump won the election fair and squa...,False,president trump win election fair square evide...,3,4,0,53,0,1,13,204,Newly Leaked Report Shows Hillary DID NOT Win ...,True
4,4,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,Katie Couric has come out publicly and corrobo...,False,katie couric come publicly corroborate story f...,16,2,1,19,0,0,18,112,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,True


# Extract Basic Lingustic Features

In [55]:
import codecs, nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import string
punctuation = set(string.punctuation)
punctuation.update("''")
punctuation.update("``")

# Create a function to extract relevant textual features
def extract_features(df):

    
    # get number of uppercase words in title
    df['upper_title'] = df['title'].apply(lambda x: len([x for x in x.split() if x.isupper() and len(x) > 2]))
    
    # get number of uppercase words in content
    df['upper_mainText'] = df['mainText'].apply(lambda x: len([x for x in x.split() if x.isupper() and len(x) > 2]))
    
    # get number of all punctuation marks in title
    df['punct_title'] = df['title'].apply(lambda x: len([char for word in x.split() for char in word if char in (punctuation)]))
    
    # get number of all punctuation marks in content
    df['punct_mainText'] = df['mainText'].apply(lambda x: len([char for word in x.split() for char in word if char in (punctuation)]))
    
    # get number of exclamation points in title and text
    df['exclamations_title'] = df['title'].apply(lambda x: x.count('!'))
    df['exclamations_text'] = df['mainText'].apply(lambda x: x.count('!'))
    
    # get number of words in title and text (excluding stopwords)
    df['title_words_not_stopword'] = df['title'].apply(lambda x: len([t for t in x.split() if t not in stop_word_list]))
    df['text_words_not_stopword'] = df['mainText'].apply(lambda x: len([t for t in x.split() if t not in stop_word_list]))
    
    return(df)

df1 = extract_features(df)

## Create Features Sets 

In [355]:
from sklearn.model_selection import train_test_split, cross_val_predict
features= [c for c in df1.columns.values if c not in ['doc_id', 'mainText', 'veracity',
                                                      'mainText_preprocessed', 'title_preprocessed',
                                                      'title_mainText_preprocessed', 'title', 'mainText_nouns','mainText_verbs' ]]
numeric_features= [c for c in df1.columns.values if c not in ['doc_id', 'title', 'title_main_text', 'mainText_preprocessed', 
                                                      'title_mainText', 'title_preprocessed', 'mainText_nouns', 
                                                      'mainText_verbs', 'title_mainText_V', 'title_mainText_N', 'hyperpartisan']]

X = df1[features]
y = df1['veracity']




In [356]:
features

['upper_title',
 'upper_mainText',
 'punct_title',
 'punct_mainText',
 'exclamations_title',
 'exclamations_text',
 'title_words_not_stopword',
 'text_words_not_stopword',
 'title_mainText',
 'hyperpartisan']

In [357]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
# Create selector class for text and numbers
class TextSelector(BaseEstimator, TransformerMixin):
    """Transformer to select a single column from the data frame to perform additional transformations on"""
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key."""
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [358]:
# Building pipelines for each feature

# Test different scalers
scaler = StandardScaler()
#scaler = MinMaxScaler()
#scaler = RobustScaler()

text = Pipeline([
                ('selector', TextSelector(key='title_mainText')),
                ('vect', TfidfVectorizer(ngram_range=(1, 2))),                
            ])

upper_title =  Pipeline([
                ('selector', NumberSelector(key='upper_title')),
                ('standard', scaler),
            ])

upper_mainText =  Pipeline([
                ('selector', NumberSelector(key='upper_mainText')),
                ('standard', scaler),
            ])

punct_title =  Pipeline([
                ('selector', NumberSelector(key='punct_title')),
                ('standard', scaler),
            ])

punct_mainText =  Pipeline([
                ('selector', NumberSelector(key='punct_mainText')),
                ('standard', scaler),
            ])


exclamations_title =  Pipeline([
                ('selector', NumberSelector(key='exclamations_title')),
                ('standard', scaler),
            ])


exclamations_text =  Pipeline([
                ('selector', NumberSelector(key='exclamations_text')),
                ('standard', scaler),
            ])


title_words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='title_words_not_stopword')),
                ('standard', scaler),
            ])

text_words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='text_words_not_stopword')),
                ('standard', scaler),
            ])

hyperpartisan =  Pipeline([
                ('selector', NumberSelector(key='hyperpartisan')),
                ('standard', scaler),
            ])

In [359]:
# Feature Union
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion([('title_mainText', text), 
                      ('upper_title', upper_title),
                      ('upper_mainText', upper_mainText),
                      ('punct_title', punct_title),
                      ('punct_mainText', punct_mainText),                    
                      ('exclamations_text', exclamations_text),
                      ('exclamations_title', exclamations_title),
                      ('title_words_not_stopword', title_words_not_stopword),
                      ('text_words_not_stopword', text_words_not_stopword),
                      ('hyperpartisan', hyperpartisan)
                     ])

feature_processing = Pipeline([('feats', feats)])



In [365]:
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression


pipeline = Pipeline([
        ('features', feats),
#        ('classifier', SVC(C=1, kernel= 'linear', max_iter= 1000, tol=0.0001, probability=True))
        ('classifier', LogisticRegression(n_jobs=1, C=1e6))
#        ('classifier', LinearSVC(C=10, max_iter = 100, penalty = 'l2', loss= 'hinge')) 
    ])





# Grid Search

In [340]:
from sklearn.model_selection import GridSearchCV

hyperparameters = {'features__title_mainText__vect__ngram_range': [(1,1), (1,2), (1,3)],
#                   'classifier__C': [1,10,100,1000],
                   'classifier__C': [0.01, 0.1, 1, 10, 100,1000,1e5,1e6],
#                   'classifier__max_iter': [100,500,1000]
                  }

clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
# Fit and tune model
clf.fit(X, y)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by Standard

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by Standard

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

...

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by Standard

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/pyt

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by Standard

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('title_mainText', Pipeline(memory=None,
     steps=[('selector', TextSelector(key='title_mainText')), ('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'features__title_mainText__vect__ngram_range': [(1, 1), (1, 2), (1, 3)], 'classifier__C': [0.01, 0.1, 1, 10, 100, 1000, 100000.0, 1000000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [341]:
clf.best_params_

{'classifier__C': 1000000.0,
 'features__title_mainText__vect__ngram_range': (1, 2)}

In [342]:
clf.best_score_

0.8276014109347443

# Cross Validation

In [366]:
from sklearn.model_selection import cross_val_score
#score = cross_val_score(text_clf, X, y, cv=10) 
score = cross_val_score(pipeline, X, y, cv=10) 
score

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by Standard

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/pyt

array([0.87719298, 0.86343612, 0.90748899, 0.85022026, 0.89427313,
       0.78854626, 0.76651982, 0.80530973, 0.80530973, 0.75663717])

In [367]:
print (score.mean())

0.831493420112208


In [368]:
from sklearn import metrics
#scores = cross_val_score(text_clf, X, y, cv=10, scoring='f1_macro')
scores = cross_val_score(pipeline, X, y, cv=10, scoring='f1_macro')
scores 

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by Standard

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/

array([0.87718353, 0.86339371, 0.90690362, 0.84916354, 0.89427108,
       0.78503788, 0.76341612, 0.80024106, 0.79834523, 0.74463277])

In [369]:
print (scores.mean())

0.8282588536058512


#  feed new data to the model

In [178]:
import numpy as np
import pandas as pd
df_new = pd.read_csv('Fake news meta/Fake_news_meta.csv')
df_new.head()

,doc_id,title,mainText,veracity,mainText_preprocessed
0,0,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,Results from a new study concluded that millio...,False,result new study conclude million could vote p...
1,1,President Trump Signed An Executive Order That...,Illegal immigrants do NOT have the same rights...,False,illegal immigrant right american citizen illeg...
2,2,New Document Reveals Lynch Instructed Comey to...,"Ah, the infamous tarmac meeting. You know…the...",False,ah infamous tarmac meeting one former attorney...
3,3,Newly Leaked Report Shows Hillary DID NOT Win ...,President Trump won the election fair and squa...,False,president trump win election fair square evide...
4,4,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,Katie Couric has come out publicly and corrobo...,False,katie couric come publicly corroborate story f...


In [141]:
len(df_new)

2268

In [33]:
# Create new field that concatenates title and text
df_new['title_mainText'] = df_new['title'] + ' ' + df_new['mainText']

In [40]:
df1_new = extract_features(df_new)
df1_new.head()

,doc_id,title,mainText,veracity,mainText_preprocessed,title_mainText,upper_title,upper_mainText,punct_title,punct_mainText,exclamations_title,exclamations_text,title_words_not_stopword,text_words_not_stopword
0,0,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,Results from a new study concluded that millio...,False,result new study conclude million could vote p...,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,10,0,1,39,1,0,13,204
1,1,President Trump Signed An Executive Order That...,Illegal immigrants do NOT have the same rights...,False,illegal immigrant right american citizen illeg...,President Trump Signed An Executive Order That...,0,3,1,17,0,0,12,108
2,2,New Document Reveals Lynch Instructed Comey to...,"Ah, the infamous tarmac meeting. You know…the...",False,ah infamous tarmac meeting one former attorney...,New Document Reveals Lynch Instructed Comey to...,0,1,1,16,0,0,9,77
3,3,Newly Leaked Report Shows Hillary DID NOT Win ...,President Trump won the election fair and squa...,False,president trump win election fair square evide...,Newly Leaked Report Shows Hillary DID NOT Win ...,3,4,0,53,0,1,13,204
4,4,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,Katie Couric has come out publicly and corrobo...,False,katie couric come publicly corroborate story f...,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,16,2,1,19,0,0,18,112


In [41]:

features_new = [c for c in df1_new.columns.values if c not in ['doc_id', 'title', 'veracity',
                                                      'mainText_preprocessed', 'title_preprocessed', 
                                                      'title_mainText_preprocessed', 'title_mainText']]
numeric_features_new = [c for c in df1_new.columns.values if c not in ['doc_id', 'title', 'main_text', 'mainText_preprocessed', 
                                                      'title_mainText', 'title_preprocessed' , 'veracity']]




In [42]:
df1_new[features]

,title_mainText,upper_title,upper_mainText,punct_title,punct_mainText,exclamations_title,exclamations_text,title_words_not_stopword,text_words_not_stopword
0,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,10,0,1,39,1,0,13,204
1,President Trump Signed An Executive Order That...,0,3,1,17,0,0,12,108
2,New Document Reveals Lynch Instructed Comey to...,0,1,1,16,0,0,9,77
3,Newly Leaked Report Shows Hillary DID NOT Win ...,3,4,0,53,0,1,13,204
4,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,16,2,1,19,0,0,18,112
5,SHOCK CLAIM: Hillary To Be Indicted On Crimina...,2,0,1,2,0,0,9,21
6,BOOM: PRESIDENT TRUMP SENT MICHELLE OBAMA A BI...,13,54,4,42,2,0,17,222
7,Obama’s Birth Brother Breaks Silence – Reveals...,0,1,0,14,0,0,10,87
8,Hillary Clinton Busted in the Middle of Huge P...,0,4,0,42,0,0,12,302
9,Breaking: Sharia Law is Gone--The House Made a...,0,0,3,38,0,0,7,194


In [43]:
# Building pipelines for each feature

# Test different scalers
scaler = StandardScaler()
#scaler = MinMaxScaler()
#scaler = RobustScaler()

text = Pipeline([
                ('selector', TextSelector(key='title_mainText')),
                ('vect', TfidfVectorizer(ngram_range=(1, 2))),                
            ])

upper_title =  Pipeline([
                ('selector', NumberSelector(key='upper_title')),
                ('standard', scaler),
            ])

upper_mainText =  Pipeline([
                ('selector', NumberSelector(key='upper_mainText')),
                ('standard', scaler),
            ])

punct_title =  Pipeline([
                ('selector', NumberSelector(key='punct_title')),
                ('standard', scaler),
            ])

punct_mainText =  Pipeline([
                ('selector', NumberSelector(key='punct_mainText')),
                ('standard', scaler),
            ])


exclamations_title =  Pipeline([
                ('selector', NumberSelector(key='exclamations_title')),
                ('standard', scaler),
            ])


exclamations_text =  Pipeline([
                ('selector', NumberSelector(key='exclamations_text')),
                ('standard', scaler),
            ])


title_words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='title_words_not_stopword')),
                ('standard', scaler),
            ])

text_words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='text_words_not_stopword')),
                ('standard', scaler),
            ])

In [44]:
# Feature Union
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion([('title_mainText', text), 
                      ('upper_title', upper_title),
                      ('upper_mainText', upper_mainText),
                      ('punct_title', punct_title),
                      ('punct_mainText', punct_mainText),                    
                      ('exclamations_text', exclamations_text),
                      ('exclamations_title', exclamations_title),
                      ('title_words_not_stopword', title_words_not_stopword),
                      ('text_words_not_stopword', text_words_not_stopword)])


In [46]:
pipeline = Pipeline([
        ('features', feats),
#        ('classifier', SVC(C=1, kernel= 'linear', max_iter= 1000, tol=0.0001, probability=True))
        ('classifier', LogisticRegression(n_jobs=1, C=1e6))
    ])

pipeline.fit(X, y)
#hyper_prediction = pipeline.predict_proba(df1_new[features])[:,1]
hyper_prediction = pipeline.predict(df1_new[features])
hyperpartisan = (hyper_prediction)
result = pd.DataFrame({'doc_id': df1_new['doc_id'], 'hyperpartisan': hyperpartisan})
complete1= pd.merge(df1_new,result,how='inner',on=['doc_id'])

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by Standard

In [38]:
complete

,doc_id,title,mainText,veracity,mainText_preprocessed,title_mainText,upper_title,upper_mainText,punct_title,punct_mainText,exclamations_title,exclamations_text,title_words_not_stopword,text_words_not_stopword,hyperpartisan
0,0,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,Results from a new study concluded that millio...,False,result new study conclude million could vote p...,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,10,0,1,39,1,0,13,204,True
1,1,President Trump Signed An Executive Order That...,Illegal immigrants do NOT have the same rights...,False,illegal immigrant right american citizen illeg...,President Trump Signed An Executive Order That...,0,3,1,17,0,0,12,108,False
2,2,New Document Reveals Lynch Instructed Comey to...,"Ah, the infamous tarmac meeting. You know…the...",False,ah infamous tarmac meeting one former attorney...,New Document Reveals Lynch Instructed Comey to...,0,1,1,16,0,0,9,77,False
3,3,Newly Leaked Report Shows Hillary DID NOT Win ...,President Trump won the election fair and squa...,False,president trump win election fair square evide...,Newly Leaked Report Shows Hillary DID NOT Win ...,3,4,0,53,0,1,13,204,False
4,4,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,Katie Couric has come out publicly and corrobo...,False,katie couric come publicly corroborate story f...,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,16,2,1,19,0,0,18,112,False
5,5,SHOCK CLAIM: Hillary To Be Indicted On Crimina...,Top investigators in the Department of Justice...,False,top investigator department justice believe am...,SHOCK CLAIM: Hillary To Be Indicted On Crimina...,2,0,1,2,0,0,9,21,False
6,6,BOOM: PRESIDENT TRUMP SENT MICHELLE OBAMA A BI...,Michelle Obama will go down in history as the ...,False,michelle obama go history first lady spend mon...,BOOM: PRESIDENT TRUMP SENT MICHELLE OBAMA A BI...,13,54,4,42,2,0,17,222,True
7,7,Obama’s Birth Brother Breaks Silence – Reveals...,Rumors have been spreading for years that Bara...,False,rumor spread year barack obama actually bear k...,Obama’s Birth Brother Breaks Silence – Reveals...,0,1,0,14,0,0,10,87,False
8,8,Hillary Clinton Busted in the Middle of Huge P...,An NBC news report claims that Hillary Clinton...,False,nbc news report claim hillary clinton secretar...,Hillary Clinton Busted in the Middle of Huge P...,0,4,0,42,0,0,12,302,False
9,9,Breaking: Sharia Law is Gone--The House Made a...,Former president Barack Obama and his Democrat...,False,former president barack obama democrat want u ...,Breaking: Sharia Law is Gone--The House Made a...,0,0,3,38,0,0,7,194,True


In [74]:
complete.to_csv("Fake news meta/FakeNews_with_hyper_BinaryLabel(SVM)_2.csv",index=False)